In this notebook, we will get the solutions for both A-priori algorithm and collaborative filtering

In [ ]:
import pandas as pd
import numpy as np

# A-priori algorithm

In [ ]:
from itertools import combinations,chain

## Write a function to find all combination of itemsets of size level-1 to generate new level-size itemsets

In [ ]:
def mingle(items, level):
    outcome = set()
    for item in items:
        for item2 in items:
            if item!=item2:
                newCombination = set()
                # If level >2, this means the itemsets contain more than 1 item
                if level>2:
                    for i in item:
                        newCombination.add(i)
                    for i in item2:
                        newCombination.add(i)
                else:
                    newCombination.add(item)
                    newCombination.add(item2)
                # Only retain combinations of itemsets that are of the size of the current level
                # The combination of larger itemsets can result in, e.g., 2-item itemsets combined 
                #                 into 4-item itemsets
                # while the level is 3, requiring 3-item itemsets
                if(len(newCombination)==level):
                    outcome.add(frozenset(newCombination))
                    
    return outcome

In [ ]:
# test the written function
assert  mingle(["a","b","c"], 2) == {frozenset({'a', 'c'}), 
                                     frozenset({'b', 'c'}), 
                                     frozenset({'a', 'b'})}

assert mingle([["a","b"],["a","c"],["a","d"]], 3) == {frozenset({'a', 'c', 'd'}), 
                                               frozenset({'a', 'b', 'd'}),
                                               frozenset({'a', 'b', 'c'})}
print("tests passed")

## Write a function that calculates the support of an itemset in a transactions database.

In [ ]:
def support(itemset,transactions,level):
    
    count = 0
    for trans in transactions:
        # Assume the transaction contains the items unless proven otherwise below
        contain = True
        # If level > 1, the itemsets contain more than 1 item, and we need to loop all items in the itemset
        if level>1:
            for item in itemset:      
                if item not in trans:
                    # No need to look further if even 1 item is not contained in the transaction
                    contain = False
                    break
        else:
            if itemset not in trans:
                contain = False
        if contain:
            count = count + 1
    return count/len(transactions)


In [ ]:
# test the written function
assert support("a", [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 1) == 0.75
assert support("d", [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 1) == 0.25
assert support(["a","b"], [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 2) == 0.5
print("tests passed")

## Write the Apriori function

In [ ]:
# for now this function will just print some results for us to observe, 
# rather than return them in a data structure

def apriori(level,transactions,items,minsup):
    
    print("\nLevel: "+str(level))
    
    # set for items with support value that is high enough
    retain = set()
    
    # find items with support value that is high enough
    for item in items:
        print(str(item)+" support: "+str(support(item,transactions,level)))      
        if support(item,transactions,level)>=minsup:
            retain.add(item)
    print("Retain: "+str(retain))
    
    level = level+1
        
    # generate new candidates
    newsets=mingle(retain,level)
    print("New itemsets: "+str(newsets))    
    
    # stop if no candidates are left or you will put all items in one set
    if len(newsets)!=0 and level<len(items)+1:
        apriori(level,transactions,newsets,minsup)

In [ ]:
apriori(1, [["a","b","c"], ["a","b","d"], ["b","c"]], {"a","b","c", "d"}, 0.6)

## Use this to run the complete algorithm.

In [ ]:
# open the data
file = open('data/baskets.csv','r')

transactions = []
items = set()

# save all transactions and items
for line in file:
    line = line.replace('\n','')
    litems = line.split(',')
    transactions.append(litems)
    for item in litems:
        items.add(item)

noItems = len(items)

# apply Apriori algorithm
apriori(1,transactions,items,0.6)   

## !!!Open Questions: can you try other "level"? How do different levels affect the final results? What does level mean?

# Collaborative filtering

Here we use the cosine similarity in order to find similar users that we can recommend products.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

## Load data

In [ ]:
# load data
ratings = pd.read_csv('data/ratings.csv')

# sample dataset
# be careful, large dataset!
ratings = ratings[:10000]

print(ratings.head())

# print some information
noMovies = len(ratings['movieId'].unique())
noUsers = len(ratings['userId'].unique())
print(str(noMovies)+" from "+str(noUsers)+' users')

## Create an empty perons_ratings matrix

In [ ]:
perons_ratings = np.zeros(shape=(noUsers,noMovies))
perons_ratings

Store movieIds as indices to use in perons_ratings matrix as the current indices don't match the sequential indices that a matrix uses.

In [ ]:
movieIds = {}
midi = 0
for value in ratings['movieId'].unique():
    movieIds[value]=midi
    midi = midi + 1

Populate the perons_ratings matrix by looping all the rows in the ratings dataframe

In [ ]:
for index, line in ratings.iterrows():
    uid = int(line['userId'])-1
    mid = movieIds[line['movieId']]
    rating = line['rating']
    # store the rating in the perons_ratings matrix at row user id - uid and column movie - mid
    perons_ratings[uid,mid]=rating
    
perons_ratings

Then we need to write two functions, one is to find similar user and the other one is to find new product!


**Note: the best solution is to write a class and you can put all global variables inside the function. However, here, to simplify the code for understanding, we write two functions and use global variables (it is not a good habit).** 

Of course, if you have interests and you're familiar with Python class, I believe you can do better by writing these functions in a class.

## Write a function to find similar users

In [ ]:
def findSimilarUsers(person_number, minCos=0.8):
    # list for similar users
    similar_users = []
    
    # for all other users
    for other_person in range(0,len(perons_ratings)-1):
        if person_number!= other_person:
            # calculate similarity
            cosine_sim = cosine(perons_ratings[person_number],perons_ratings[other_person])

            # retain other user if similarity threshold is met
            if cosine_sim>minCos:
                similar_users.append(perons_ratings[other_person])
    print("#similar users: "+str(len(similar_users)))
    return similar_users

## Write a function to find new product

In [ ]:
def findNewProducts(similar_users,person_number, minScore=1.3):
    if len(similar_users)>0:
        # celli stands for the column number of the perons_ratings matrix, i.e., a movie
        for movie_number in range(len(perons_ratings[person_number])-1):
            # if there is no rating for our current user, calculate new score
            if perons_ratings[person_number,movie_number]==0:
                other_scores = 0
                
                # add scores of similar users
                for other in similar_users:
                    other_scores += other[movie_number]
                                 
                # store average score 
                average_score = other_scores/len(similar_users)
#                 print(f"average_score: {average_score} \t {other_scores} / {len(similar_users)} ")
                
                # if the score is greater than a threshold, e.g. 1.3 (on scale from 0 to 5)
                # (it's so low, because most people did not rate most movies)
                if average_score>minScore:
                    print(f'Recommendation for user {person_number} is a movie {movie_number}, score {average_score}')
#     

## Complete the experiment

In [ ]:
# minimum cosine similarity
minCos = 0.8
# minimum score
minScore = 1.3

for row_number in range(0,len(perons_ratings)-1):
    print("\nFinding recommendations for user "+str(row_number))
    simmilarUsers = findSimilarUsers(row_number, minCos)
    findNewProducts(simmilarUsers,row_number, minScore)

## !!!Open Questions: Can you think of a approach to visualise the final results for better understanding?

Hints: for example, Directed Graph!